### 1. General info of dataset GSE147766

This is the Jupyter Notebook for dataset GSE147766. Its dataset includes an overall big txt file. As seen below, in the csv file, each row is a gene and each column is a cell.

Thus, we need to transform this txt file and generate an overall AnnData object for all samples. 

GSM4445608	NB01 (RNA-seq)

GSM4445609	NB02 (RNA-seq)

GSM4445610	NB09 (RNA-seq)

GSM4445611	NB11 (RNA-seq)

GSM4445612	NB12 (RNA-seq)

GSM4445613	NB13 (RNA-seq)

GSM4445614	NB14 (RNA-seq)

GSM4445615	NB16 (RNA-seq)

GSM4445616	NB17 (RNA-seq)

GSM4445617	NB18 (RNA-seq)

GSM4445618	NB19 (RNA-seq)

GSM4445619	NB20 (RNA-seq)

GSM4445620	NB21 (RNA-seq)

GSM4445621	NB22 (RNA-seq)

GSM4445622	NB23 (RNA-seq)

GSM4445623	NB24 (RNA-seq)

GSM4445624	NB26 (RNA-seq)

GSM6058205	NB34 (RNA-seq)

GSM6058206	NB37 (RNA-seq)

In [60]:
# Environment setup
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata
import scipy

In [62]:
# inspect the dataset
input = pd.read_csv("/scratch/user/s4543064/xiaohan-john-project/data/GSE147766/GSM4445608_NB01.count.csv.gz", sep=',', index_col=0) # the first column contains gene names and is the index

print(input.head())
print(input.shape) # (33868 genes, 949 cells)

             NB01_AAACCTGTCGTAGATC-1  NB01_AAACGGGAGTGTACTC-1  \
MIR1302-2HG                        0                        0   
FAM138A                            0                        0   
OR4F5                              0                        0   
AL627309.1                         0                        0   
AL627309.3                         0                        0   

             NB01_AAACGGGAGTGTCCCG-1  NB01_AAACGGGCAGTTCATG-1  \
MIR1302-2HG                        0                        0   
FAM138A                            0                        0   
OR4F5                              0                        0   
AL627309.1                         0                        0   
AL627309.3                         0                        0   

             NB01_AAAGATGAGCTTCGCG-1  NB01_AAAGCAACAAGTCTAC-1  \
MIR1302-2HG                        0                        0   
FAM138A                            0                        0   
OR4F5                  

As shown above, the dataset contains 4058 cells and 23686 genes.

### 2. Overall AnnData object of the dataset

<span style="color:red">**IMPORTANT:**</span> transpose the DataFrame.values to match the AnnData.X

1. `DataFrame.columns`: cell barcodes, which go into `.obs`
2. `DataFrame.index`: gene names, `.var`
3. `DataFrame.values`: the transpose of the expression matrix, `.X`

In [6]:
from pathlib import Path

In [13]:
input = pd.read_csv("xiaohan-john-project/data/GSE147766_RAW/"+sample_name+'.csv', sep=',', index_col=0)

In [17]:
pd.DataFrame(index=input.index)

""
MIR1302-2HG
FAM138A
OR4F5
AL627309.1
AL627309.3
...
AC233755.2
AC233755.1
AC240274.1
AC213203.2


In [67]:
# Specify directory paths
data_directory = Path('/scratch/user/s4543064/xiaohan-john-project/data/GSE147766/')
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE147766/')

# Loop through all files in the directory
for sample_directory in data_directory.iterdir():
    sample_name = sample_directory.stem
    sample_h5ad = sample_name + '_uni.h5ad'

    input = pd.read_csv("/scratch/user/s4543064/xiaohan-john-project/data/GSE147766/"+sample_name+'.gz', sep=',', index_col=0)

    matrix = scipy.sparse.csr_matrix(input.values.T)
    obs_name = pd.DataFrame(index=input.columns)
    var_name = pd.DataFrame(index=input.index)

    sample = anndata.AnnData(X=matrix, obs=obs_name, var=var_name)

    # Create an observation metric info to store related features
    obs_metrics = pd.DataFrame(index=sample.obs_names) ## Get the identifiers
    obs_metrics['cancer_type'] = 'Neuroblastoma '
    obs_metrics['dataset'] = 'GSE147766'
    obs_metrics['tissue'] = 'Adrenal'
    obs_metrics['sample_barcode'] = sample_name
    obs_metrics['uni_barcode'] = obs_metrics['dataset'] + '_' + obs_metrics.index.astype(str)
    
    sample.obs = obs_metrics
    sample.obs.set_index("uni_barcode", drop=False, inplace=True)
    print(sample)

    # save the anndata object
    output_path = write_directory.joinpath(sample_h5ad)
    sample.write_h5ad(output_path, compression="gzip")

AnnData object with n_obs × n_vars = 1608 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 8347 × 32966
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 949 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 1361 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 17287 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 3217 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 10267 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode', 'uni_barcode'
AnnData object with n_obs × n_vars = 6623 × 33868
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode

In [68]:
sample.obs

,cancer_type,dataset,tissue,sample_barcode,uni_barcode
uni_barcode,,,,,
GSE147766_NB20_AAACGGGAGCGATCCC-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_AAACGGGAGCGATCCC-1
GSE147766_NB20_AAACGGGAGCTAAGAT-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_AAACGGGAGCTAAGAT-1
GSE147766_NB20_AAACGGGGTCCTAGCG-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_AAACGGGGTCCTAGCG-1
GSE147766_NB20_AAAGATGCAAACAACA-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_AAAGATGCAAACAACA-1
GSE147766_NB20_AAAGCAAAGGTGTTAA-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_AAAGCAAAGGTGTTAA-1
...,...,...,...,...,...
GSE147766_NB20_TTGGCAACAGTGAGTG-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_TTGGCAACAGTGAGTG-1
GSE147766_NB20_TTTACTGAGGTGGGTT-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_TTTACTGAGGTGGGTT-1
GSE147766_NB20_TTTATGCAGTGGGATC-1,Neuroblastoma,GSE147766,Adrenal,GSM4445619_NB20.count.csv,GSE147766_NB20_TTTATGCAGTGGGATC-1


In [69]:
sample.var

""
MIR1302-2HG
FAM138A
OR4F5
AL627309.1
AL627309.3
...
AC233755.2
AC233755.1
AC240274.1
AC213203.2


In [70]:
sample.X[10:1000, 10:2000].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

### 3. Confirmation of created AnnData object

In [2]:
output = '/QRISdata/Q6104/Xiaohan/2_AnnData_objs/GSE147766/GSM4445624_NB26.count_uni.h5ad'
sample = anndata.read_h5ad(output)
print(sample)

AnnData object with n_obs × n_vars = 5288 × 33833
    obs: 'cancer_type', 'dataset', 'tissue', 'sample_barcode'


In [3]:
sample.var

""
MIR1302-2HG
FAM138A
OR4F5
AL627309.1
AL627309.3
...
MT-ND4L
MT-ND4
MT-ND5
MT-ND6


In [4]:
sample.obs

,cancer_type,dataset,tissue,sample_barcode
uni_barcode,,,,
GSE147766_NB26_AAACCTGAGATATGGT-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
GSE147766_NB26_AAACCTGAGGATGTAT-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
GSE147766_NB26_AAACCTGAGTCCATAC-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
GSE147766_NB26_AAACCTGAGTGCGTGA-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
GSE147766_NB26_AAACCTGAGTGGGATC-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
...,...,...,...,...
GSE147766_NB26_TTTGTCAGTACCGGCT-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
GSE147766_NB26_TTTGTCAGTTGGACCC-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count
GSE147766_NB26_TTTGTCAGTTTCGCTC-1,Neuroblastoma,GSE147766,Adrenal,GSM4445624_NB26.count


### 4. Convert AnnData objects to SingleCellExperiment objects

In [73]:
from pathlib import Path

import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects import r
from rpy2.robjects.conversion import localconverter

# Specify directory paths
write_directory = Path('/scratch/user/s4543064/xiaohan-john-project/write/GSE147766')

# Loop through all files in the directory
for file in write_directory.iterdir():
    sample_name = file.stem
    if "_uni.h5ad" in file.name:
        sample_anndata = anndata.read_h5ad(file)
        sample_anndata.X = sample_anndata.X.astype('float32')
        sample_sce_file = sample_name + ".rds"

        with localconverter(anndata2ri.converter):
            sample_sce = anndata2ri.py2rpy(sample_anndata)
        print(sample_sce)
        
        # Save the sce object in .rds file
        robjects.globalenv["sample_sce"] = sample_sce
        sample_sce_path = write_directory / sample_sce_file
        robjects.r("saveRDS(sample_sce, file='{}')".format(sample_sce_path))

class: SingleCellExperiment 
dim: 33868 2973 
metadata(0):
assays(1): X
rownames(33868): MIR1302-2HG FAM138A ... AC213203.2 AC213203.1
rowData names(0):
colnames(2973): GSE147766_NB24_AAACCTGAGATGTTAG-1
  GSE147766_NB24_AAACCTGCAAGCTGTT-1 ...
  GSE147766_NB24_TTTGTCATCCTCAACC-1 GSE147766_NB24_TTTGTCATCGGAGCAA-1
colData names(5): cancer_type dataset tissue sample_barcode uni_barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

class: SingleCellExperiment 
dim: 33868 949 
metadata(0):
assays(1): X
rownames(33868): MIR1302-2HG FAM138A ... AC213203.2 AC213203.1
rowData names(0):
colnames(949): GSE147766_NB01_AAACCTGTCGTAGATC-1
  GSE147766_NB01_AAACGGGAGTGTACTC-1 ...
  GSE147766_NB01_TTTGTCACACTTACGA-1 GSE147766_NB01_TTTGTCAGTGCCTGTG-1
colData names(5): cancer_type dataset tissue sample_barcode uni_barcode
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

class: SingleCellExperiment 
dim: 33868 1075 
metadata(0):
assays(1): X
rownames(33868): MIR1302-2HG FAM138A ... AC213203.